In [16]:
import numbers

class IntegerField:
    def __init__(self, min_, max_):
        self._min = min_
        self._max = max_

    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
    
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError(f'{self.prop_name} must be an integer value.')
        if value < self._min:
            raise ValueError(f'{self.prop_name} must be >= {self._min}')
        if value > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max}')
        instance.__dict__[self.prop_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name, None)

In [17]:
class Person:
    age = IntegerField(0, 100)

In [18]:
p = Person()

In [19]:
p.age =5

In [20]:
print(p.age)

5


In [21]:
try:
    p.age = 200
except ValueError as ex:
    print(ex)

age must be <= 100


In [22]:
import unittest

In [23]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)

In [26]:
class TestIntegerField(unittest.TestCase):
    class Person:
        age = IntegerField(0, 10)

    def test_set_age_ok(self):
        p = self.Person()
        p.age = 0
        self.assertEqual(0, p.age)

In [27]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [28]:
class TestIntegerField(unittest.TestCase):
    class Person:
        age = IntegerField(0, 10)

    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        self.Person.age = IntegerField(5, 10)

        p = self.Person()
        p.age = 5
        self.assertEqual(5, p.age)

In [29]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ERROR

ERROR: test_set_age_ok (__main__.TestIntegerField)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\nikit\AppData\Local\Temp\ipykernel_39664\1869443481.py", line 11, in test_set_age_ok
    p.age = 5
  File "C:\Users\nikit\AppData\Local\Temp\ipykernel_39664\104829885.py", line 18, in __set__
    instance.__dict__[self.prop_name] = value
AttributeError: 'IntegerField' object has no attribute 'prop_name'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


In [30]:
class TestIntegerField(unittest.TestCase):
    class Person:
        age = IntegerField(0, 10)

    def create_person(self, min_, max_):
        self.Person.age = IntegerField(min_, max_)
        self.Person.age.__set_name__(Person, "age")
        return self.Person()

    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        p = self.create_person(min_, max_)
        p = self.Person()
        
        p.age = 5
        self.assertEqual(5, p.age)

In [31]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [32]:
class Person():
    a= 10

In [33]:
type(Person)

type

In [34]:
Person = type("Person", (), {'a' : 10})

In [35]:
Person.__dict__

mappingproxy({'a': 10,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [36]:
Person.a

10

In [39]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'age' : IntegerField(min_, max_)})
        return obj

    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        p = self.create_test_class(min_, max_)
        
        p.age = 5
        self.assertEqual(5, p.age)

In [40]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [41]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'age' : IntegerField(min_, max_)})
        return obj

    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_values = range(min_, max_ + 1)

        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)

       

In [42]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [45]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'age': IntegerField(min_, max_)})
        return obj()
        
    def test_set_age_ok(self):
        """Tests that valid values can be assigned/retrieved"""
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_values = range(min_, max_)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
                
    def test_set_age_invalid(self):
        """Tests that invalid values raise ValueErrors"""
        min_ = -10
        max_ = 10
        obj = self.create_test_class(min_, max_)
        bad_values = list(range(min_ - 5, min_))
        bad_values += list(range(max_ + 1, max_ + 5))
        bad_values += [10.5, 1 + 0j, 'abc', (1, 2)]
        
        for i, value in enumerate(bad_values):
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    obj.age = value
                    
    def test_class_get(self):
        """Tests that class attribute retrieval returns the descriptor instance"""
        obj = self.create_test_class(0, 0)
        obj_class = type(obj)
        self.assertIsInstance(obj_class.age, IntegerField)

In [46]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField)
Tests that class attribute retrieval returns the descriptor instance ... ok
test_set_age_invalid (__main__.TestIntegerField)
Tests that invalid values raise ValueErrors ... ok
test_set_age_ok (__main__.TestIntegerField)
Tests that valid values can be assigned/retrieved ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.008s

OK


In [53]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'age': IntegerField(min_, max_)})
        return obj()
        
    def test_set_age_ok(self):
        """Tests that valid values can be assigned/retrieved"""
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_values = range(min_, max_)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
                
    def test_set_age_invalid(self):
        """Tests that invalid values raise ValueErrors"""
        min_ = -10
        max_ = 10
        obj = self.create_test_class(min_, max_)
        bad_values = list(range(min_ - 5, min_))
        bad_values += list(range(max_ + 1, max_ + 5))
        bad_values += [10.5, 1 + 0j, 'abc', (1, 2)]
        
        for i, value in enumerate(bad_values):
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    obj.age = value
                    
    def test_class_get(self):
        """Tests that class attribute retrieval returns the descriptor instance"""
        obj = self.create_test_class(0, 0)
        obj_class = type(obj)
        self.assertIsInstance(obj_class.age, IntegerField)

    def test_set_age_min_only(self):
        """Tests that we can specify min value only"""
        min_ = 0
        max_ = None
        obj = self.create_test_class(min_, max_)
        values = range(min_, min_+100, 10)

        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)

    def test_set_age_max_only(self):
        """Tests that we can specify max value only"""
        min_ = None
        max_ = 10
        obj = self.create_test_class(min_, max_)
        values = range(max_-100, max_, 10)

        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
    
    def test_set_age_no_limits(self):
        """Tests that we can specify max value only"""
        min_ = None
        max_ = None
        obj = self.create_test_class(min_, max_)
        values = range(-100, 100, 10)

        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)

In [54]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField)
Tests that class attribute retrieval returns the descriptor instance ... ok
test_set_age_invalid (__main__.TestIntegerField)
Tests that invalid values raise ValueErrors ... ok
test_set_age_max_only (__main__.TestIntegerField)
Tests that we can specify max value only ... ok
test_set_age_min_only (__main__.TestIntegerField)
Tests that we can specify min value only ... ok
test_set_age_no_limits (__main__.TestIntegerField)
Tests that we can specify max value only ... ok
test_set_age_ok (__main__.TestIntegerField)
Tests that valid values can be assigned/retrieved ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.010s

OK


In [55]:
class IntegerField:
    def __init__(self, min_ = None, max_ = None):
        self._min = min_
        self._max = max_

    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
    
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError(f'{self.prop_name} must be an integer value.')
        if self._min is not None and value < self._min:
            raise ValueError(f'{self.prop_name} must be >= {self._min}')
        if self._max is not None and value > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max}')
        instance.__dict__[self.prop_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name, None)

In [56]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField)
Tests that class attribute retrieval returns the descriptor instance ... ok
test_set_age_invalid (__main__.TestIntegerField)
Tests that invalid values raise ValueErrors ... ok
test_set_age_max_only (__main__.TestIntegerField)
Tests that we can specify max value only ... ok
test_set_age_min_only (__main__.TestIntegerField)
Tests that we can specify min value only ... ok
test_set_age_no_limits (__main__.TestIntegerField)
Tests that we can specify max value only ... ok
test_set_age_ok (__main__.TestIntegerField)
Tests that valid values can be assigned/retrieved ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.007s

OK


In [80]:
class CharField:
    def __init__(self, min_=None, max_=None):
        min_ = min_ or 0  
        min_ = max(min_, 0) 
        self._min = min_
        self._max = max_

    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
    
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.prop_name} must be a string.')
        if self._min is not None and len(value) < self._min:
            raise ValueError(f'{self.prop_name} must be >= {self._min} chars.')
        if self._max is not None and len(value) > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max} chars')
        instance.__dict__[self.prop_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

In [81]:
class Person:
    name = CharField(1, 10)


In [82]:
p = Person()

In [84]:
try:
    p.name = "fghvhkbhjkyukv"
except ValueError as ex:
    print(ex)

name must be <= 10 chars


In [85]:
p.name = 'Nik'

In [87]:
p.name

'Nik'

In [88]:
class Person:
    name = CharField(1)

In [89]:
p = Person()

In [90]:
p.name = "ftvgyuhoildewvedor8iu7fgvbiewrdy8gvfb"

In [91]:
class TestCharField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'name': CharField(min_, max_)})
        return obj()
        
    def test_set_name_ok(self):
        """Tests that valid values can be assigned/retrieved"""
        min_ = 1
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_lengths = range(min_, max_)
        
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
            
    def test_set_name_invalid(self):
        """Tests that invalid values raise ValueErrors"""
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        bad_lengths = list(range(min_ - 5, min_))
        bad_lengths += list(range(max_ + 1, max_ + 5))
        for i, length in enumerate(bad_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    obj.name = value
                    
    def test_class_get(self):
        """Tests that class attribute retrieval returns the descriptor instance"""
        obj = self.create_test_class(0, 0)
        obj_class = type(obj)
        self.assertIsInstance(obj_class.name, CharField)
        
    def test_set_name_min_only(self):
        """Tests that we can specify a min length only"""
        min_ = 0
        max_ = None
        obj = self.create_test_class(min_, max_)
        valid_lengths = range(min_, min_ + 100, 10)
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
    
    def test_set_name_min_negative_or_none(self):
        """Tests that setting a negative or None length results in a zero length"""
        obj = self.create_test_class(-10, 100)
        self.assertEqual(type(obj).name._min, 0)
        self.assertEqual(type(obj).name._max, 100)
        
        obj = self.create_test_class(None, None)
        self.assertEqual(type(obj).name._min, 0)
        self.assertIsNone(type(obj).name._max)
        
    def test_set_name_max_only(self):
        """Tests that we can specify a max length only"""
        min_ = None
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_lengths = range(max_ - 100, max_, 10)
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
                
    def test_set_name_no_limits(self):
        """Tests that we can use CharField without any limits at all"""
        min_ = None
        max_ = None
        obj = self.create_test_class(min_, max_)
        valid_lengths = range(0, 100, 10)
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)

In [92]:
run_tests(TestCharField)

test_class_get (__main__.TestCharField)
Tests that class attribute retrieval returns the descriptor instance ... ok
test_set_name_invalid (__main__.TestCharField)
Tests that invalid values raise ValueErrors ... ok
test_set_name_max_only (__main__.TestCharField)
Tests that we can specify a max length only ... ok
test_set_name_min_negative_or_none (__main__.TestCharField)
Tests that setting a negative or None length results in a zero length ... ok
test_set_name_min_only (__main__.TestCharField)
Tests that we can specify a min length only ... ok
test_set_name_no_limits (__main__.TestCharField)
Tests that we can use CharField without any limits at all ... ok
test_set_name_ok (__main__.TestCharField)
Tests that valid values can be assigned/retrieved ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.011s

OK
